In [1]:
#chack for updated files\n,
%load_ext autoreload
%autoreload 2
#import packages
from __future__ import division

import numpy as np
import glob, json
import pickle
import os as os_pack
import matplotlib.pyplot as plt
import corner
#%matplotlib inline\n",
%config InlineBackend.figure_format = 'retina'
import healpy as hp
import os, glob, json, pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as sl
import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const
from enterprise_extensions import blocks
from enterprise_extensions import models as ee_models
from enterprise_extensions import model_utils as ee_model_utils
from enterprise_extensions import model_orfs
from enterprise_extensions.frequentist import optimal_statistic as opt_stat
from enterprise_extensions import sampler as ee_sampler
from enterprise.signals.signal_base import LogLikelihood
import enterprise_wavelets as models
from enterprise.signals.deterministic_signals import Deterministic
from enterprise.signals.parameter import function
from la_forge.core import Core
from la_forge.diagnostics import plot_chains
from la_forge import rednoise
import la_forge
import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
    #style

In [56]:
#Loading in pickle and noise files
pint_pickle = '/home/reyna/OS_15yr/15yr_data/v1p1_de440_pint_bipm2019.pkl'
noise_file = '/home/reyna/OS_15yr/15yr_data/v1p1_wn_dict.json'
#psrlist = np.loadtxt('/home/reyna/15yr_v1p0/15yr_v1-20211001T235643Z-001/15yr_v1/psrlist_15yr_pint.txt', dtype = str)
with open(noise_file, 'r') as h:
    noise_params = json.load(h)
with open(pint_pickle,'rb') as f:
    allpsrs = pickle.load(f)
psrs = []
for ii,p in enumerate(allpsrs):
    psrs.append(p)
#Temporary to get code to not crash
psrs = psrs[0:8]
psrlist = [psr.name for psr in psrs]
print(psrlist)
for i in range(len(psrs)):
    print((max(psrs[i].toas) - min(psrs[i].toas))/(3.17*10**(7)))

['B1855+09', 'B1937+21', 'B1953+29', 'J0023+0923', 'J0030+0451', 'J0340+4130', 'J0406+3039', 'J0437-4715']
15.515060297068947
15.802872967471835
11.08458774068679
8.97542529762611
15.460752938661816
8.097196263848735
3.5471797714338695
4.73231500896138


In [57]:
glitches = []
log10_f0 = parameter.Uniform(np.log10(3.5e-9), np.log10(1e-7))("Glitch_"+str(1)+'_'+'log10_f0')
phase0 = parameter.Uniform(0, 2*np.pi)("Glitch_"+str(1)+'_'+'phase0')
tau = parameter.Uniform(0.2, 5)("Glitch_"+str(1)+'_'+'tau')
t0 = parameter.Uniform(0.0, 10.0)("Glitch_"+str(1)+'_'+'t0')
psr_idx = parameter.Uniform(-0.5, len(psrs)-0.5)("Glitch_"+str(1)+'_'+'psr_idx')
log10_h = parameter.LinearExp(-6.5, -5)("Glitch_"+str(1)+'_'+'log10_h')
glitch_wf = models.glitch_delay(log10_h = log10_h, tau = tau, log10_f0 = log10_f0, t0 = t0, phase0 = phase0, tref=53000*86400,
                                        psr_float_idx = psr_idx, pulsars=psrs)
glitches.append(deterministic_signals.Deterministic(glitch_wf, name='Glitch'+str(1) ))

In [58]:
tm = gp_signals.TimingModel(use_svd=True)
wn = blocks.white_noise_block(vary=False, inc_ecorr=False)
#s = base_model
s = tm + wn + glitches[0]
model = []
for p in psrs:
    model.append(s(p))
with open(noise_file, 'r') as fp:
    noisedict = json.load(fp)
    pta = signal_base.PTA(model)
    pta.set_default_params(noisedict)

In [59]:
d0 = parameter.sample(pta.params)
x0 = np.array([d0[par.name] for par in pta.params])

In [60]:
pta.get_lnlikelihood(x0)

729928.481336031

In [61]:
726231.1702248305
724566.2053894639

448209.3360017643

In [62]:
d0

{'Glitch_1_log10_f0': -7.756878408525551,
 'Glitch_1_log10_h': -5.264166700004374,
 'Glitch_1_phase0': 2.7528575556558974,
 'Glitch_1_psr_idx': -0.253086923663032,
 'Glitch_1_t0': 6.913254541763757,
 'Glitch_1_tau': 2.0302139073586076}

In [63]:
import Fast_Burst_likelihood as FB

In [64]:
FB1 = FB.FastBurst(pta = pta, psrs = psrs, params = d0, Npsr = len(psrs), tref=53000*86400)
#FB2 = FB.FastBurst(pta = pta, psrs = psrs, params = d0, Npsr = len(psrs), tref=53000*86400)

Nvecs arary:  [[1.69852389e-12 1.12772641e-10 3.26328092e-12 ... 4.85655692e-12
 5.14669141e-13 1.79238956e-12], [6.95185145e-14 1.57784257e-13 2.75345324e-14 ... 1.37317696e-13
 6.53510393e-13 3.42658995e-13], [4.94236756e-10 4.03404993e-10 7.90600715e-10 ... 2.03923984e-10
 4.69194849e-10 4.02505715e-10], [1.78150121e-13 6.22555077e-13 1.41224754e-13 ... 4.44123286e-11
 2.71409648e-11 3.12885720e-11], [4.61695300e-12 5.66353407e-11 6.83697178e-12 ... 9.39287544e-12
 1.01635764e-11 6.60318228e-12], [3.58685902e-11 2.77047195e-11 8.77407399e-11 ... 4.00248661e-11
 7.61997117e-11 4.82576810e-11], [2.30848299e-11 2.25095542e-12 6.90732193e-12 ... 1.26507575e-11
 3.65281122e-11 1.93290456e-11], [5.49729188e-12 4.98164427e-12 5.77752577e-12 ... 4.40088221e-12
 4.86261323e-12 1.46668814e-09], ...]
-23685595.61118301
logdet_phi:  15289.165017480458
logdet_sigma:  4448.603620972837
dotSigmaTNr:  6171.411084360541
logdet_phi:  25604.746234093782
logdet_sigma:  8639.86438247935
dotSigmaTNr:  48

In [65]:
FB1.get_lnlikelihood(10**(x0[1]), x0[2], 10**(x0[0]), (3.15*10**7)*x0[5], (3.15*10**7)*x0[4], x0[3])

Amplitude:  5.442936905362693e-06
Frequency:  1.750336670043231e-08
glitch_index:  -0.253086923663032
Old Sigma:  [0. 0.]
New sigma:  [-5.03683514e-06 -2.06297210e-06]
Old M and N:  [[0. 0.]
 [0. 0.]] [0. 0.]
Input time:  2520.4573850180363
Cosine:  [-2.37361327e-05 -2.37361327e-05 -2.37361327e-05 ... -6.99858185e-11
 -6.99858185e-11 -6.99858185e-11]
Sine:  [-1.96108530e-04 -1.96108530e-04 -1.96108530e-04 ...  1.11103219e-10
  1.11103219e-10  1.11103219e-10]
108185601493995.8
-6997592417447.869
-6997592417447.804
138110284558319.05
MM matrix: [[[ 1.08185601e+14 -6.99759242e+12]
  [-6.99759242e+12  1.38110285e+14]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]]

 [[ 0.0

724566.2053894639

In [12]:
482127.23089051904 #Bence version

482127.23089051904

In [13]:
448629.5379522237 #Enterprise version

448629.5379522237

In [14]:
LogL:  447940.2057596576
LogL:  447934.2907498001
LogL:  447934.2692285934
LogL:  447934.6153235128
447934.6153235128

447934.6153235128